In [1]:
import torch
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F

# Load the processed label data
df = pd.read_csv('processed_label_data.csv')

# Load the tensors
tensor1 = torch.load(df.path[0])
tensor2 = torch.load(df.path[1])

# Stack the tensors to create a batch of input data
input_data = torch.stack((tensor1, tensor2))

# Print the shape of the input data
print("Input data shape:", input_data.shape)

# Load the labels
labels = df[['positive', 'neutral', 'negative']]
labels = torch.tensor(labels.values[:2, :])

class ComplexNN(nn.Module):
    def __init__(self, cnn_input_channels, cnn_output_channels, cnn_kernel_size, cnn_stride, 
                 cnn_pool_kernel_size, cnn_pool_stride, hidden_size, output_size):
        super(ComplexNN, self).__init__()
        
        # CNN layer
        self.cnn = nn.Conv1d(cnn_input_channels, cnn_output_channels, cnn_kernel_size, stride=cnn_stride)
        self.pool = nn.MaxPool1d(cnn_pool_kernel_size, stride=cnn_pool_stride)
        
        # Fully connected layers
        self.input_size = cnn_output_channels * ((1500 - cnn_kernel_size) // cnn_pool_kernel_size + 1) # Update input_size
        self.hidden = nn.Linear(self.input_size, hidden_size)
        self.relu = nn.ReLU()
        
        # Output layer without softmax
        self.output = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Apply CNN and max pooling
        x = self.cnn(x)
        x = self.pool(x)
        
        # Flatten the output of CNN
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = self.hidden(x)
        x = self.relu(x)
        
        # Output layer
        x = self.output(x)
        
        # Apply softmax to get probabilities
        x = F.softmax(x, dim=1)
        return x

# Create an instance of the ComplexNN model
cnn_input_channels = 128  # Update the input channels
cnn_output_channels = 16
cnn_kernel_size = 5
cnn_stride = 1
cnn_pool_kernel_size = 2
cnn_pool_stride = 2
num_data_points = 1500
hidden_size = 128
output_size = 3

# Update the input_channels in the model definition
model = ComplexNN(cnn_input_channels, cnn_output_channels, cnn_kernel_size, cnn_stride,
                   cnn_pool_kernel_size, cnn_pool_stride, hidden_size, output_size)

# Print the model architecture
print(model)

# Forward pass through the model
output = model(input_data)

# Print the output shape
print("Output shape:", output.shape)

# Print the probabilities for the first data point in the batch
print("Probabilities for the first data point:")
print(output)


Input data shape: torch.Size([2, 128, 1500])
ComplexNN(
  (cnn): Conv1d(128, 16, kernel_size=(5,), stride=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (hidden): Linear(in_features=11968, out_features=128, bias=True)
  (relu): ReLU()
  (output): Linear(in_features=128, out_features=3, bias=True)
)
Output shape: torch.Size([2, 3])
Probabilities for the first data point:
tensor([[0.3512, 0.3468, 0.3020],
        [0.2856, 0.3643, 0.3502]], grad_fn=<SoftmaxBackward0>)
